In [39]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")
from nltk import sent_tokenize, word_tokenize
nltk.download('punkt')

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [40]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --. Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [41]:
# Load and clean the data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [42]:
# Parse the cleaned novels. This can take some time.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [43]:
# Group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one DataFrame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [44]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True)


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,abide,ability,able,abominate,abroad,absence,absent,absolute,absolutely,absurd,abuse,accept,acceptable,acceptance,accession,accident,accidentally,accommodate,accommodation,accompany,accomplish,accomplishment,accord,accordingly,account,accuse,acknowledge,acknowledgement,acquaint,acquaintance,acquainted,acquiescence,acquire,acre,act,action,active,activity,actual,actually,...,wishing,wit,withdraw,withstand,witness,woman,wonder,wonderful,wonderland,wood,word,work,world,worldly,worth,worthy,wound,wow,wretched,wretchedness,wriggle,write,writing,wrong,wrought,yard,yarmouth,yawn,ye,year,yer,yes,yesterday,yield,young,youth,zeal,zealous,text,author
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alice begin tired sit sister bank have twice p...,Carroll
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.224454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,remarkable Alice think way hear Rabbit oh dear,Carroll
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,shall late,Carroll


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

KeyboardInterrupt: ignored

---
2. In the 2-grams example above, you only used 2-grams as your features. This time, use both 1-grams and 2-grams together as your feature set. Run the same models as in the example and compare the results.

In [ ]:
vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(1,2))


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

In [ ]:
Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

---
1. Converting words or sentences into numeric vectors is fundamental when working with text data. To make sure that you have a solid handle on how these vectors work, generate the TF-IDF vectors for the last three sentences of the example from the beginning of this checkpoint (from the BoW revisited: TF-IDF section).

In [47]:
doc_string = r"The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing. I would rather put strawberries on my ice cream for dessert; they have the best taste. The taste of caramel is a fantastic accompaniment to tasty mint ice cream."
doc = text_cleaner(doc_string)
doc

"The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing. I would rather put strawberries on my ice cream for dessert; they have the best taste. The taste of caramel is a fantastic accompaniment to tasty mint ice cream."

In [48]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(doc)
doc

The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing. I would rather put strawberries on my ice cream for dessert; they have the best taste. The taste of caramel is a fantastic accompaniment to tasty mint ice cream.

In [49]:
# Group into sentences
sents = [sent for sent in doc.sents]
sents



[The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing.,
 I would rather put strawberries on my ice cream for dessert; they have the best taste.,
 The taste of caramel is a fantastic accompaniment to tasty mint ice cream.]

In [74]:
sentences = pd.Series(sents)
sentences

0    (The, Lumberjack, Song, is, the, funniest, Mon...
1    (I, would, rather, put, strawberries, on, my, ...
2    (The, taste, of, caramel, is, a, fantastic, ac...
dtype: object

In [53]:
# sentences[0].is_punct

AttributeError: ignored

In [75]:
def clean_data(sentences):
    return " ".join([token.lemma_ for token in sentences if not token.is_punct and not token.is_stop]).strip()

sentences = sentences.apply(
    clean_data
    )

sentences

0    Lumberjack Song funniest Monty Python bit thin...
1              strawberry ice cream dessert good taste
2    taste caramel fantastic accompaniment tasty mi...
dtype: object

In [72]:
for sent in sentences:
    print(sent)

Lumberjack Song funniest Monty Python bit think laugh
strawberry ice cream dessert good taste
taste caramel fantastic accompaniment tasty mint ice cream


In [58]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i in range(len(sentences)):
    sentences[i] = " ".join([token.lemma_ for token in sentences[i] if not token.is_punct and not token.is_stop])
    
for sent in sentences:
    print(sent)

Lumberjack Song funniest Monty Python bit think laugh
strawberry ice cream dessert good taste
taste caramel fantastic accompaniment tasty mint ice cream


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()


# Applying the vectorizer
X = vectorizer.fit_transform(sentences)

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
# display(tfidf_df)
sentences = pd.concat([tfidf_df, sentences], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,accompaniment,bit,caramel,cream,dessert,fantastic,funniest,good,ice,laugh,lumberjack,mint,monty,python,song,strawberry,taste,tasty,think,0
0,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.353553,0.353553,0.000000,0.353553,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553,Lumberjack Song funniest Monty Python bit thin...
1,0.000000,0.000000,0.000000,0.349498,0.459548,0.000000,0.000000,0.459548,0.349498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.459548,0.349498,0.000000,0.000000,strawberry ice cream dessert good taste
2,0.385323,0.000000,0.385323,0.293048,0.000000,0.385323,0.000000,0.000000,0.293048,0.000000,0.000000,0.385323,0.000000,0.000000,0.000000,0.000000,0.293048,0.385323,0.000000,taste caramel fantastic accompaniment tasty mi...


4: 1.585, 1, 0, 1, 1.585, 0,0,0,0

5: 0,0,0,0,0, .585, 1, 1.585, 1

6: 0,0,0,0,0,0, 1, 0, 2